# Make multiwavelength light curves using archival data
***

## Learning Goals    
By the end of this tutorial, you will be able to:
 - automatically load a catalog of sources
 - automatically search NASA and non-NASA resources for light curves
 - store light curves in a Pandas multiindex dataframe
 - plot all light curves on the same plot
 
 
## Introduction:
 - A user has a sample of interesting targets for which they would like to see a plot of available archival light curves.  We start with a small set of changing look AGN from Yang et al., 2018, which are automatically downloaded. Changing look AGN are cases where the broad emission lines appear or disappear (and not just that the flux is variable). 
 - We model light curve plots after van Velzen et al. 2021.  We search through a curated list of time-domain NASA holdings as well as non-NASA sources.  HEASARC catalogs used are Fermi and Beppo-Sax, IRSA catalogs used are ZTF and WISE, and MAST catalogs used are Pan-Starrs, TESS, Kepler, and K2.  Non-NASA sources are Gaia and IceCube. This list is generalized enough to include many types of targets to make this notebook interesting for many types of science.  All of these time-domain archives are searched in an automated fashion using astroquery or APIs.
 - Light curve data storage is a tricky problem.  Currently we are using a multi-index Pandas dataframe, as the best existing choice for right now.  One downside is that we need to manually track the units of flux and time instead of relying on an astropy storage scheme which would be able to do some of the units worrying for us (even astropy can't do all magnitude to flux conversions).  Astropy does not currently have a good option for multi-band light curve storage.
 - We intend to explore a ML classifier for these changing look AGN light curves.
 
## Input:
 - choose from a list of known changing look AGN from the literature
 
  OR - 
 - input your own sample

## Output:
 - an archival optical + IR + neutrino light curve
 
## Non-standard Imports:
- `astroquery` to interface with archives APIs
- `astropy` to work with coordinates/units and data structures
- `lightkurve` to search TESSS, Kepler, and K2 archives
- `urllib` to handle archive searches with website interface
- `acstools` to work with HST magnitude to flux conversion
- `unTimely` to retrieve WISE light curves
- `alerce` to convert ZTF object names into coordinates

## Authors:
Jessica Krick, Shoubaneh Hemmati, Andreas Faisst, Troy Raen, Brigitta Sipocz, Dave Shupe

## Acknowledgements:
Suvi Gezari, Antara Basu-zych,Stephanie LaMassa\
MAST, HEASARC, & IRSA Fornax teams



In [1]:
#ensure all dependencies are installed
!pip install -r requirements.txt

You should consider upgrading via the '/opt/conda/bin/python3 -m pip install --upgrade pip' command.


In [12]:
import sys
import time
import warnings
from math import ceil
from multiprocessing import Pool

import astropy.units as u
import matplotlib.pyplot as plt
import numpy as np
from astropy.table import Table
from scipy import stats
import pandas as pd

warnings.filterwarnings('ignore')

#get tools for accessing Meisner et al., 2023 WISE light curves
try:
    from unTimely_Catalog_tools import unTimelyCatalogExplorer
except ImportError:
    !pip install git+https://github.com/fkiwy/unTimely_Catalog_explorer
    !pip install reproject
    from unTimely_Catalog_tools import unTimelyCatalogExplorer

# local code imports
sys.path.append('code/')
from panstarrs import panstarrs_get_lightcurves
from gaia_functions import Gaia_get_lightcurve
from HCV_functions import HCV_get_lightcurves
from icecube_functions import icecube_get_lightcurve
from sample_selection import get_lamassa_sample, get_macleod16_sample, get_ruan_sample, get_macleod19_sample, get_sheng_sample, \
    get_green_sample, get_lyu_sample, get_lopeznavas_sample, get_hon_sample, get_yang_sample,  get_SDSS_sample, clean_sample
from ztf_functions import ZTF_get_lightcurve
from data_structures import MultiIndexDFObject
from heasarc_functions import HEASARC_get_lightcurves
from TESS_Kepler_functions import TESS_Kepler_get_lightcurves
from WISE_functions import WISE_get_lightcurves

## 1. Define the Sample
 We define here a "gold" sample of spectroscopically confirmed changing look AGN and quasars. This sample includes both objects which change from type 1 to type 2 and also the opposite.  Future studies may want to treat these as seperate objects or seperate QSOs from AGN.
 
 Bibcodes for the samples used are listed next to their functions for reference.  
 
 Functions used to grab the samples from the papers use Astroquery, NED, SIMBAD, Vizier, and in a few cases grab the tables from the html versions of the paper.

In [3]:
#build up the sample
coords =[]
labels = []

#choose your own adventure:

#get_lamassa_sample(coords, labels)  #2015ApJ...800..144L
#get_macleod16_sample(coords, labels) #2016MNRAS.457..389M
#get_ruan_sample(coords, labels) #2016ApJ...826..188R
#get_macleod19_sample(coords, labels)  #2019ApJ...874....8M
#get_sheng_sample(coords, labels)  #2020ApJ...889...46S
#get_green_sample(coords, labels)  #2022ApJ...933..180G
#get_lyu_sample(coords, labels)  #z32022ApJ...927..227L
#get_lopeznavas_sample(coords, labels)  #2022MNRAS.513L..57L
#get_hon_sample(coords, labels)  #2022MNRAS.511...54H
get_yang_sample(coords, labels)   #2018ApJ...862..109Y

#now get some "normal" QSOs for use in the classifier
#there are ~500K of these, so choose the number based on
#a balance between speed of running the light curves and whatever 
#the ML algorithms would like to have

# num_normal_QSO = 100 
#get_SDSS_sample(coords, labels, num_normal_QSO)

# remove duplicates and attach an objectid to the coords
coords_list, labels_list = clean_sample(coords, labels)

#add an index-like list with our own object name
object_names = [['CLAGN' + str(i)] for i in range(len(coords_list))]

## For testing
#coords_list = coords_list[0:4]
#labels_list = labels_list[0:4]
#object_names = object_names[0:4]

Changing Look AGN- Yang et al:  31
after duplicates removal, sample size: 30


### 1.1 Build your own Sample

To build your own sample, you can follow the examples of functions above to grab coordinates from your favorite literature resource, 

or

You can use [astropy's read](https://docs.astropy.org/en/stable/io/ascii/read.html) function to read in an input table
and then convert that table into a list of [skycoords](https://docs.astropy.org/en/stable/api/astropy.coordinates.SkyCoord.html) 

## 2. Find light curves for these targets in NASA catalogs
  - We search a curated list of time-domain catalogs from all NASA astrophysics archives


In [4]:
### Initialize Pandas MultiIndex data frame for storing the light curves
df_lc = MultiIndexDFObject()

### 2.1 HEASARC: FERMI & Beppo SAX


In [ ]:
start_serial = time.time()
mission_list = ['FERMIGTRIG', 'SAXGRBMGRB']
radius = 0.1*u.degree

#go out and find all light curves in the above curated list which match our target positions
df_lc_heasarc = HEASARC_get_lightcurves(coords_list,labels_list,radius, mission_list)

df_lc.append(df_lc_heasarc)

### 2.2 IRSA: ZTF

In [ ]:
df_lc_ZTF = ZTF_get_lightcurve(coords_list,labels_list,plotprint=0) ## number of plots to show to be set by plotprint

#add the resulting dataframe to all other archives
df_lc.append(df_lc_ZTF)

### 2.3 IRSA:WISE

- use the unTimely catalog which ties together all WISE & NEOWISE 2010 - 2020 epochs.  Specifically it combined all observations at a single epoch to achieve deeper mag limits than individual observations alone.
- https://github.com/fkiwy/unTimely_Catalog_explorer
- https://iopscience-iop-org.caltech.idm.oclc.org/article/10.3847/1538-3881/aca2ab

In [ ]:
bandlist = ['w1', 'w2']
WISE_radius = 1.0

df_lc_WISE = WISE_get_lightcurves(coords_list, labels_list, WISE_radius, bandlist)

#add the resulting dataframe to all other archives
df_lc.append(df_lc_WISE)

### 2.4 MAST: Pan-STARRS
Query the Pan-STARRS API; based on this [example](https://ps1images.stsci.edu/ps1_dr2_api.html)

In [ ]:
#Do a panstarrs search
panstarrs_radius = 1.0/3600.0    # search radius = 1 arcsec
df_lc_panstarrs = panstarrs_get_lightcurves(coords_list, labels_list, panstarrs_radius)

#add the resulting dataframe to all other archives
df_lc.append(df_lc_panstarrs)

### 2.5 MAST: Asteroid Terrestrial-impact Last Alert System (ATLAS)
 - All-sky stellar reference catalog 
 -  MAST hosts this catalog but there are three barriers to using it
     1. it is unclear if the MAST [holdings]( https://archive.stsci.edu/hlsp/atlas-refcat2#section-a737bc3e-2d56-4827-9ab4-838fbf8d67c1) include the individual epoch photometry and 
     2. it is only accessible with casjobs, not through python notebooks.  
     3. magnitude range (g, r, i) < 19mag makes it not relevant for this use case
 
One path forward if this catalog becomes scientifically interesting is to put in a MAST helpdesk ticket to see if 1) they do have the light curves, and 2) they could switch the catalog to a searchable with python version.  There are some ways of [accessing casjobs with python](<https://github.com/spacetelescope/notebooks/blob/master/notebooks/MAST/HSC/HCV_CASJOBS/HCV_casjobs_demo.ipynb), but not this particular catalog.  
 

### 2.6 MAST: TESS, Kepler and K2
 - use [`lightKurve`](https://docs.lightkurve.org/index.html) to search all 3 missions and download light curves
 


In [ ]:
#go get the lightcurves using lightkurve
TESS_radius = 1.0  #arcseconds
df_lc_TESS = TESS_Kepler_get_lightcurves(coords_list, labels_list, TESS_radius)

#add the resulting dataframe to all other archives
df_lc.append(df_lc_TESS)

### 2.7 MAST: HCV
 - [hubble catalog of variables](https://archive.stsci.edu/hlsp/hcv) 
 - using [this notebook](https://archive.stsci.edu/hst/hsc/help/HCV/HCV_API_demo.html) as a reference to search and download light curves via API

In [ ]:
%%time
#Do an HCV search
HCV_radius = 1.0/3600.0 # radius = 1 arcsec
df_lc_HCV = HCV_get_lightcurves(coords_list, labels_list, HCV_radius)

#add the resulting dataframe to all other archives
df_lc.append(df_lc_HCV)

## 3. Find light curves for these targets in relevant, non-NASA catalogs


### 3.1 Gaia 


In [ ]:
starttime = time.time()
## Note: This is very slow. Can probably make faster with direct SQL search?
df_lc_gaia = Gaia_get_lightcurve(coords_list,  labels_list , object_names , verbose = 1)

#add the resulting dataframe to all other archives
df_lc.append(df_lc_gaia)

print('gaia search took:', time.time() - starttime, 's')

### 3.2 ASAS-SN (all sky automated survey for supernovae) 
- Has a [website](https://asas-sn.osu.edu/photometry) that can be manually searched; but no API which would allow automatic searches from within this notebook
- Magnitude range of this survey is not consistent with the magnitude range of our CLAGN.  If this catalog becomes scientifically interesting, one path forward would be to ask ASAS-SN team about implementing an API



### 3.3 Icecube Neutrinos

There are several [catalogs](https://icecube.wisc.edu/data-releases/2021/01/all-sky-point-source-icecube-data-years-2008-2018) (basically one for each year of IceCube data from 2008 - 2018). The following code creates a large catalog by combining
all the yearly catalogs.
The IceCube catalog contains Neutrino detections with associated energy and time and approximate direction (which is uncertain by half-degree scales....). Usually, for active events only one or two Neutrinos are detected, which makes matching quite different compared to "photons". For our purpose, we will list the top 3 events in energy that are within a given distance to the target.

This time series (time vs. neutrino energy) information is similar to photometry. We choose to storing time and energy in our data structure, leaving error = 0. What is __not__ stored in this format is the distance or angular uncertainty of the event direction. 

In [ ]:
%%time
## Note: IceCube catalogs are currently stored in a personal Caltech Box directory of A. Faisst
df_lc_icecube = icecube_get_lightcurve(coords_list , labels_list , object_names ,
                                   icecube_select_topN = 3,
                                   path = "./data/",
                                   verbose = 1)

#add the resulting dataframe to all other archives
df_lc.append(df_lc_icecube)
end_serial = time.time()

In [ ]:
#benchmarking
print('total time for serial archive calls is ', end_serial - start_serial, 's')

## 4. Parallel Processing the archive calls


In [5]:
# define some variables in case the above serial cells are not run
mission_list = ["FERMIGTRIG", "SAXGRBMGRB"]
heasarc_radius = 0.1 * u.degree
bandlist = ["w1", "w2"]
wise_radius = 1.0
panstarrs_radius = 1.0 / 3600.0  # search radius = 1 arcsec
lk_radius = 1.0  # arcseconds
hcv_radius = 1.0 / 3600.0  # radius = 1 arcsec

In [6]:
# calculate the number of workers needed
n_single_archives, n_multiple_archives = 6, 2  # must sum to total number of archives called
n_chunks_per_archive = 5  # will make one api call per chunk per slow archive
n_workers = n_single_archives + n_multiple_archives * n_chunks_per_archive

In [7]:
parallel_starttime = time.time()

# start a multiprocessing pool and run all the archive queries
parallel_df_lc = MultiIndexDFObject()  # to collect the results
callback = parallel_df_lc.append  # will be called once on the result returned by each archive
with Pool(processes=n_workers) as pool:

    # start the processes that call the fast archives
    pool.apply_async(
        Gaia_get_lightcurve, (coords_list, labels_list, object_names, 1), callback=callback
    )
    pool.apply_async(
        HEASARC_get_lightcurves, (coords_list, labels_list, heasarc_radius, mission_list), callback=callback
    )
    pool.apply_async(
        HCV_get_lightcurves, (coords_list, labels_list, hcv_radius), callback=callback
    )
    pool.apply_async(
        icecube_get_lightcurve, (coords_list, labels_list, object_names, 3, "./data/", 1), callback=callback
    )
    pool.apply_async(
        panstarrs_get_lightcurves, (coords_list, labels_list, panstarrs_radius), callback=callback
    )
    pool.apply_async(
        TESS_Kepler_get_lightcurves, (coords_list, labels_list, lk_radius), callback=callback
    )

    # split coords_list into smaller chunks and call slow archives
    chunksize = ceil(len(coords_list) / n_chunks_per_archive)  # num coords per api call
    for n in range(0, len(coords_list), chunksize):
        coords = coords_list[n : n + chunksize]

        # start the processes that call the slow archives
        pool.apply_async(
            WISE_get_lightcurves, (coords, labels_list, wise_radius, bandlist), callback=callback
        )
        pool.apply_async(
            ZTF_get_lightcurve, (coords, labels_list, 0), callback=callback
        )

    pool.close()  # signal that no more jobs will be submitted to the pool
    pool.join()  # wait for all jobs to complete, including the callback

parallel_endtime = time.time()

  3%|▎         | 1/30 [00:01<00:45,  1.55s/it]

Unzipping IceCube data |  

 17%|█▋        | 1/6 [00:03<00:15,  3.08s/it]]

0 no matches


 10%|█         | 3/30 [00:03<00:37,  1.37s/it]

...

No data found for target "<SkyCoord (ICRS): (ra, dec) in deg
    (236.3735, 25.19107)>".
ERROR:lightkurve.search:No data found for target "<SkyCoord (ICRS): (ra, dec) in deg
    (236.3735, 25.19107)>".
 87%|████████▋ | 26/30 [02:31<00:19,  4.98s/it]

./353/3539p000/3539p000_w2_e011.cat.fits.gz
./221/2211p287/2211p287_w2_e016.cat.fits.gz
./353/3539p000/3539p000_w1_e012.cat.fits.gz


100%|██████████| 6/6 [02:32<00:00, 25.37s/it]


./353/3539p000/3539p000_w2_e012.cat.fits.gz
./353/3539p000/3539p000_w1_e013.cat.fits.gz
./353/3539p000/3539p000_w2_e013.cat.fits.gz
./353/3539p000/3539p000_w1_e014.cat.fits.gz
./353/3539p000/3539p000_w2_e014.cat.fits.gz
./353/3539p000/3539p000_w1_e015.cat.fits.gz
./353/3539p000/3539p000_w2_e015.cat.fits.gz
./353/3539p000/3539p000_w1_e016.cat.fits.gz
./353/3539p000/3539p000_w2_e016.cat.fits.gz


100%|██████████| 6/6 [02:36<00:00, 26.00s/it]
No data found for target "<SkyCoord (ICRS): (ra, dec) in deg
    (237.57179, 41.65064)>".
ERROR:lightkurve.search:No data found for target "<SkyCoord (ICRS): (ra, dec) in deg
    (237.57179, 41.65064)>".
 90%|█████████ | 27/30 [02:38<00:16,  5.51s/it]No data found for target "<SkyCoord (ICRS): (ra, dec) in deg
    (238.24292, 27.62456)>".
ERROR:lightkurve.search:No data found for target "<SkyCoord (ICRS): (ra, dec) in deg
    (238.24292, 27.62456)>".
 93%|█████████▎| 28/30 [02:43<00:10,  5.37s/it]No data found for target "<SkyCoord (ICRS): (ra, dec) in deg
    (238.66774, 36.4978)>".
ERROR:lightkurve.search:No data found for target "<SkyCoord (ICRS): (ra, dec) in deg
    (238.66774, 36.4978)>".
 97%|█████████▋| 29/30 [02:49<00:05,  5.62s/it]No data found for target "<SkyCoord (ICRS): (ra, dec) in deg
    (354.01242, 0.29132)>".
ERROR:lightkurve.search:No data found for target "<SkyCoord (ICRS): (ra, dec) in deg
    (354.01242, 0.29132)>".
1

In [8]:
# look at the results
print('parallel processing took', parallel_endtime - parallel_starttime, 's')
parallel_df_lc.data

parallel processing took 173.46000123023987 s


flux       err
objectid label   band time                            
18       Yang 18 zg   58202.331875  0.027848  0.003577
                      58227.298484  0.036612  0.003902
                      58227.300370  0.033007  0.003784
                      58229.262442  0.028886  0.003624
                      58230.331678  0.027161  0.003545
...                                      ...       ...
29       Yang 18 w2   58285.304688  0.132997   0.02059
                      58447.750000  0.130770  0.016078
                      58649.484375  0.148172  0.019403
                      58811.941406  0.162735  0.016916
                      59177.519531  0.203871  0.021772

[29880 rows x 2 columns]

In [18]:
# # Save the data for future use with ML notebook
parquet_savename = 'data/df_lc_051123.parquet.gzip'
#parallel_df_lc.data.to_parquet(parquet_savename)
#print("file saved!")

file saved!


In [19]:
# could load a previously saved file in order to plot
#parallel_df_lc = MultiIndexDFObject()
#parallel_df_lc.data = pd.read_parquet(parquet_savename)
#print("file loaded!")

file loaded!


## 5. Make plots of luminosity as a function of time
- model plots after [van Velzen et al., 2021](https://arxiv.org/pdf/2111.09391.pdf)


In [ ]:
%%time

for objectid, coord in coords_list:
    
    ## Set up =================
    # choose whether to plot data from the serial or parallel calls
    singleobj = df_lc.data.loc[objectid]
    # singleobj = parallel_df_lc.data.loc[objectid]

    # Set up for plotting. We use the "mosaic" method so we can plot
    # the ZTF data in a subplot for better visibility.
    fig, axes = plt.subplot_mosaic(mosaic=[["A"],["A"],["B"]] , figsize=(10,8))
    plt.subplots_adjust(hspace=0.3 , wspace=0.3)

    ## Plot all the bands in the *main plot* (A) ====================
    leg_handles_A = []
    max_list = [] # store maximum flux for each band
    ztf_minmax_tab = Table(names=["tmin","tmax","fluxmin","fluxmax"]) # store the min and max of the ZTF band fluxes and time
    has_ztf = False # flag to set to True if ZTF data is available.
    has_icecube = False # flag to set to True if IceCube data is available.
    
    for band in singleobj.index.unique('band'):
        
        # get data
        band_lc = singleobj.loc[:, band, :]
        band_lc.reset_index(inplace = True)

        # first clean dataframe to remove erroneous rows
        band_lc_clean = band_lc[band_lc['time'] < 65000]
        
        # Do some sigma-clipping, but only if more than 10 data points.
        if len(band_lc_clean) >= 10:
            band_lc_clean = band_lc_clean[np.abs(stats.zscore(band_lc_clean.flux)) < 3.0]
        
        # before plotting need to scale the Kepler, K2, and TESS fluxes to the other available fluxes
        if band in ['Kepler', 'K2', 'TESS']: # Note: these are not included anymore...
            
            #remove outliers in the dataset
            #bandlc_clip = band_lc_clean[(np.abs(stats.zscore(band_lc_clean['flux'])) < 3.0)]

            #find the maximum value of 'other bands'
            max_electrons = max(band_lc_clean.flux)
            factor = np.mean(max_list)/ max_electrons
            lh = axes["A"].errorbar(bandlc_clip.time, bandlc_clip.flux * factor, bandlc_clip.err* factor,
                                    capsize = 3.0,label = band)
            
        # ZTF is special because we are plotting the data also in "B" zoom-in
        elif band in ['zg','zr','zi']: # for ZTF
            has_ztf = True
            max_list.append(max(band_lc_clean.flux)) 
            lh = axes["A"].errorbar(band_lc_clean.time, band_lc_clean.flux, band_lc_clean.err,
                                    capsize = 1.0, elinewidth=0.5,marker='o',markersize=2,linestyle='', label = "ZTF {}".format(band))
            ztf_minmax_tab.add_row( [np.min(band_lc_clean.time) , np.max(band_lc_clean.time) , np.min(band_lc_clean.flux) , np.max(band_lc_clean.flux) ] )
            
            
            # plot ZTF in zoomin
            p1 = axes["B"].errorbar(band_lc_clean.time, band_lc_clean.flux, band_lc_clean.err,
                                    capsize = 1.0, elinewidth=0.5, marker='o',linestyle='',markersize=0.5, alpha=0.5,
                                    label = "ZTF {}".format(band) , color=lh.lines[0].get_color())
            
            
            
            # overplot running mean fo ZTF in zoomin 
            xx = band_lc_clean.time.values # Note: need to use .values here to remove indexing.
            yy = band_lc_clean.flux.values # Note: need to use .values here to remove indexing.
            ee = band_lc_clean.err.values # Note: need to use .values here to remove indexing.
            x_bin = 30 # in MJD
            x_grid = np.arange(np.nanmin(xx) , np.nanmax(xx)+x_bin/4 , x_bin/4)
            tmp = Table(names=["xbin","ybin","yerr"])
            
            for xxx in x_grid:
                s = np.where( np.abs(xx - xxx) < x_bin/2 )[0]
                if len(s) > 1:
                    mn = np.nansum(yy[s]*ee[s]) / np.nansum(ee[s]) # weighted mean
                    tmp.add_row([xxx , mn , np.nanstd(yy[s])])
                else:
                    tmp.add_row([xxx , np.nan , np.nan])
                    
            axes["B"].plot(tmp["xbin"] , tmp["ybin"] , "-", linewidth=1.5 , color=p1.lines[0].get_color())
            
            
        
        # IceCube is special because it's only events (= limits on plot.)
        elif band in ["IceCube"]:
            has_icecube = True
            # We deal with this later. Need to wait for all the things to plot
            # so we know the y limits.
            
        # Now plot everything else
        else:
            max_list.append(max(band_lc_clean.flux)) 
            lh = axes["A"].errorbar(band_lc_clean.time, band_lc_clean.flux, band_lc_clean.err,
                                    capsize = 3.0, label = band)
            
        leg_handles_A.append(lh) # add legend handles
        
    ## Now plot IceCube.
    # we had to wait for all the data to be plotted so we know
    # the y-limits of the resulting final plot. So, we do IceCube
    # at the end.
    if has_icecube:
        band_lc = singleobj.loc[:, "IceCube", :]
        band_lc.reset_index(inplace = True)
        band_lc_clean = band_lc[band_lc['time'] < 65000]
        
        y = axes["A"].get_ylim()[0] + np.diff(axes["A"].get_ylim())*0.7
        dy = np.diff(axes["A"].get_ylim())/20
        lh = axes["A"].errorbar(band_lc_clean.time , np.repeat(y , len(band_lc_clean.time)) , yerr=dy, uplims=True ,
                                fmt="o"  , label="IceCube" , color="black")
    
        leg_handles_A.append(lh) # add legend handles (for IceCube)
    
    
    ## Do Axes ===============
    axes["A"].set_ylabel('Flux(mJy)')
    
    # Plot the ZTF bands in a separate plot to show their variability
    # more clearly. Can still also plot the rest, just change the x and
    # y axis limits. Only do this if ZTF is available for source.
    if has_ztf:
        axes["B"].set_ylabel('Flux(mJy)')
        axes["B"].set_xlabel('Time(MJD)')
        axes["B"].set_xlim( np.min(ztf_minmax_tab["tmin"])-100 , np.max(ztf_minmax_tab["tmax"])+100 )
    
    
    plt.legend(handles=leg_handles_A , bbox_to_anchor=(1.4,3.5))
    plt.tight_layout()
    #save the plot to data/*.pdf
    savename = "data/lightcurve_{}.pdf".format(objectid)
    plt.savefig(savename, bbox_inches="tight")
    plt.show()

## References

This work made use of:

- Astroquery; Ginsburg et al., 2019, 2019AJ....157...98G

- Astropy; Astropy Collaboration 2022, Astropy Collaboration 2018, Astropy Collaboration 2013, 2022ApJ...935..167A, 2018AJ....156..123A, 2013A&A...558A..33A

- Lightkurve; Lightkurve Collaboration 2018, 2018ascl.soft12013L

- acstools; https://zenodo.org/record/7406933#.ZBH1HS-B0eY

- unTimely; Meisner et al., 2023, 2023AJ....165...36M

- Alerce; Forster et al., 2021, 2021AJ....161..242F